# Homework 3 - Using DP for object reconstruction from shadows

In this homework we use mitsuba studio 3 (3.6.4), with python 3.12. 

In [1]:
import mitsuba as mi
import os
import drjit as dr
import numpy as np
from tqdm import tqdm

# See the variants available on the Mac M3 Pro
mi.variants()

['scalar_rgb',
 'scalar_spectral',
 'scalar_spectral_polarized',
 'llvm_ad_rgb',
 'llvm_ad_mono',
 'llvm_ad_mono_polarized',
 'llvm_ad_spectral',
 'llvm_ad_spectral_polarized']

In [2]:
# We set the LLVM AutoDiff - MacOS
mi.set_variant('llvm_ad_rgb')

We open a 3D scene in Mitsuba XML format and render it.

In [3]:
# This is some macos specific - DRJIT LLVM lib path, which needs to be exported
os.environ['DRJIT_LIBLLVM_PATH'] = '/opt/homebrew/opt/llvm/lib/libLLVM.dylib'

sphere_scene = mi.load_file('sphere-scene.xml')
sphere_img = mi.render(sphere_scene, spp=16)

We can view the image with the Bitmap class

In [4]:
mi.util.convert_to_bitmap(sphere_img)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [900, 800],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 2.06 MiB of image data ]
]

We also open our reference scene

In [5]:
cube_scene = mi.load_file('cube-scene.xml')
cube_img = mi.render(cube_scene, spp=16)

mi.util.convert_to_bitmap(cube_img)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [900, 800],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 2.06 MiB of image data ]
]

We can traverse our scene to find the shadow object cast on the wall, this is the object against which we will optimize.

In [6]:
params = mi.traverse(sphere_scene)
print(params)

SceneParameters[
  --------------------------------------------------------------------------------------------
  Name                                     Flags    Type           Parent
  --------------------------------------------------------------------------------------------
  default-bsdf.brdf_0.reflectance.value    ∂        Float          UniformSpectrum
  elm__1.near_clip                                  float          PerspectiveCamera
  elm__1.far_clip                                   float          PerspectiveCamera
  elm__1.shutter_open                               float          PerspectiveCamera
  elm__1.shutter_open_time                          float          PerspectiveCamera
  elm__1.film.size                                  ScalarVector2u HDRFilm
  elm__1.film.crop_size                             ScalarVector2u HDRFilm
  elm__1.film.crop_offset                           ScalarPoint2u  HDRFilm
  elm__1.x_fov                             ∂, D     Float          Pers

In [7]:
# Get the shadows from setting up the sensors in the correct spots
dist = 5
sensor = mi.load_dict({
    'type': 'perspective',
    'id': 'sphere_shadow_sensor',
    'fov_axis': 'x',
    'fov': 115,
    'principal_point_offset_x': 0.0,
    'principal_point_offset_y': 0.0,
    'near_clip': 0.1,
    'far_clip': 900.0,
    'to_world': mi.ScalarTransform4f().rotate(mi.ScalarPoint3f(1, 0, 0), 179)
                                    .rotate(mi.ScalarPoint3f(0, 1, 0), 0)
                                    .rotate(mi.ScalarPoint3f(0, 0, 1), 180) @
                mi.ScalarTransform4f().translate(mi.ScalarPoint3f(0.0, 1.833394, 0.182561)),
    'sampler': {
        'type': 'independent',
        'sample_count': 16
    },
    'film': {
        'type': 'hdrfilm',
        'sample_border': True,
        'width': 700,
        'height': 600
    }
})

In [8]:
image_sphere_shadow = mi.render(sphere_scene, sensor=sensor, spp=64)
mi.Bitmap(image_sphere_shadow)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [700, 600],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 4.81 MiB of image data ]
]

In [9]:
image_cube_shadow = mi.render(cube_scene, sensor=sensor, spp=64)
bitmap_ref = mi.Bitmap(image_cube_shadow)
bitmap_ref

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [700, 600],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 4.81 MiB of image data ]
]

We now set up the optimizer and the optimization loop

In [ ]:
params.keep(['sphere.vertex_positions', 'sphere.faces'])

integrator = mi.load_dict({
    'type': 'direct_projective',
    'sppi': 1024,
    'sppc': 0,
    'sppp': 0,
})

lambda_ = 25
ls = mi.ad.LargeSteps(params['sphere.vertex_positions'], params['sphere.faces'], lambda_)

opt = mi.ad.Adam(lr=1e-1, uniform=True)
opt['u'] = ls.to_differential(params['sphere.vertex_positions'])

# Optimization loop
for i in tqdm(range(10)):
    params['sphere.vertex_positions'] = ls.from_differential(opt['u'])
    params.update()
    
    sphere_shadow = mi.render(sphere_scene, params=params, sensor=sensor, integrator=integrator, spp=16, seed=i)
    
    loss = dr.mean(dr.square(sphere_shadow - image_cube_shadow))
    dr.backward(loss)

    opt.step()
    
    if i % 10 == 0:
        mi.util.write_bitmap(f"progress_{i:04d}.png", sphere_shadow)

final_sphere_img = mi.render(sphere_scene, params=params, spp=256)
mi.util.write_bitmap("final_optimized.png", final_sphere_img)

final_diff = dr.abs(final_sphere_img - mi.render(cube_scene, spp=256))
mi.util.write_bitmap("final_difference.png", final_diff)


  2%|▏         | 3/200 [01:02<1:02:55, 19.17s/it]

2025-03-30 20:23:53 WARN main [projective.py:510] init_indirect_silhouette_octree(): Failed to build the Octree guiding distribution! No guiding distibution for indirect visibility discontinuities will be used.
2025-03-30 20:23:53 WARN main [projective.py:510] The original error message from the octree construction:
2025-03-30 20:23:53 WARN main [projective.py:510] ​[distr_1d.h:220] DiscreteDistribution: empty distribution!

  2%|▏         | 4/200 [01:17<57:25, 17.58s/it]  

2025-03-30 20:24:08 WARN main [projective.py:510] init_indirect_silhouette_octree(): Failed to build the Octree guiding distribution! No guiding distibution for indirect visibility discontinuities will be used.
2025-03-30 20:24:08 WARN main [projective.py:510] The original error message from the octree construction:
2025-03-30 20:24:08 WARN main [projective.py:510] ​[distr_1d.h:220] DiscreteDistribution: empty distribution!

  4%|▎         | 7/200 [02:02<50:37, 15.74s/it]

...

2025-03-30 21:13:11 WARN main [projective.py:510] init_indirect_silhouette_octree(): Failed to build the Octree guiding distribution! No guiding distibution for indirect visibility discontinuities will be used.
2025-03-30 21:13:11 WARN main [projective.py:510] The original error message from the octree construction:
2025-03-30 21:13:11 WARN main [projective.py:510] ​[distr_1d.h:220] DiscreteDistribution: empty distribution!

 48%|████▊     | 97/200 [1:00:08<2:14:14, 78.20s/it]

2025-03-30 21:22:59 WARN main [projective.py:510] init_indirect_silhouette_octree(): Failed to build the Octree guiding distribution! No guiding distibution for indirect visibility discontinuities will be used.
2025-03-30 21:22:59 WARN main [projective.py:510] The original error message from the octree construction:
2025-03-30 21:22:59 WARN main [projective.py:510] ​[distr_1d.h:220] DiscreteDistribution: empty distribution!

 50%|█████     | 100/200 [1:01:09<1:01:09, 36.69s/it]


KeyboardInterrupt: 

In [11]:
print(params['sphere.vertex_positions'])

[-0.00672302, 1.9716, -0.0378031, .. 26409 skipped .., -0.0109039, 1.97155, -0.0432505]
